In [1]:
import pandas as pd
import glob
import numpy as np
import os
import smartsheet
import datetime


file_ex = 'xlsx'
file_csv = '.csv'

# Central

In [2]:
county= ['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER', 'TUOLUMNE', 'MENDOCINO', 'TULARE', 'KERN']

Central_county = ['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER']
SCentral_couty = ['TUOLUMNE', 'MENDOCINO', 'TULARE', 'KERN']


# smartsheets Shet IDs
Central_id = '1844440261257092'
South_id= '6292262490531716'

# Central's API
Central = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') #API

Central.errors_as_exceptions(True)

# Central's API
South = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API
South.errors_as_exceptions(True)

# # Download Smartrsheets to 
# Central.Sheets.get_sheet_as_excel(Central_id, download_path=r"C:\Users\jacque.trahan\Downloads")
# South.Sheets.get_sheet_as_excel(South_id, download_path=r"C:\Users\jacque.trahan\Downloads") 

# Function to read in column titles and row vales and make out data frames
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Central_SS = smartsheet_loadup(Central.Sheets.get_sheet(Central_id))
South_SS = smartsheet_loadup(South.Sheets.get_sheet(South_id))



# North

In [3]:
# read in BOP Smart hseets tracker
path1 = r"C:\Users\jacque.trahan\Downloads\BOP_North Div 2021 Debris Removal Tracker*"
load_it1 = glob.glob(path1 + file_ex)

df_B = pd.read_excel(max(load_it1, key=os.path.getctime), parse_dates=True)


# read in ANVIL Smart sheeets tracker
path_s = r"C:\Users\jacque.trahan\Downloads\ANVIL_North Div 2021 Debris Removal Tracker*"
load_it_s = glob.glob(path_s + file_ex)

df_A = pd.read_excel(max(load_it_s, key=os.path.getctime), parse_dates=True)

C:\Users\jacque.trahan\AppData\Roaming\Python\Python39\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\jacque.trahan\AppData\Roaming\Python\Python39\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# RTR data

In [4]:
# set up counties for each project
Central_county = ['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER']
SCentral_county = ['TUOLUMNE', 'MENDOCINO', 'TULARE', 'KERN']


BOP_county = ['PLUMAS', 'SISKIYOU', 'LASSEN', 'TRINITY']

Anvil_county = ['SHASTA', 'TRINITY', 'SISKIYOU']

In [39]:
path2 = r"C:\Users\jacque.trahan\Downloads\aDataManagerTicketExport*"
load2 = glob.glob(path2 + file_ex)

df_RTR = pd.read_excel(max(load2, key=os.path.getctime), parse_dates=True, na_values=None)

RTR = df_RTR.copy()
RTR['Service Code'] = RTR['Service Code'].str.strip().str.upper()
RTR = RTR[(RTR['Service Code'].isin(['12A', '12B'])) & (RTR['Is Void'] != True)] 



In [44]:
RTR['End Time'] = pd.to_datetime(RTR['End Time']).dt.date
RTR['Start Time'] = pd.to_datetime(RTR['Start Time']).dt.date

RTR['Total Days on Property'] = (RTR['End Time'] - RTR['Start Time']).dt.days + 1

RTR['Average days on Property'] = (C_RTR['Ticket Date'].size())
RTR[['End Time','Start Time','Total Days on Property']]

,End Time,Start Time,Total Days on Property
5348,2021-12-06,2021-12-06,1
5379,2021-12-06,2021-12-06,1
5380,2021-12-06,2021-12-06,1
5382,2021-12-06,2021-12-06,1
5383,2021-12-06,2021-12-06,1
...,...,...,...
54594,2022-05-05,2022-05-05,1
54596,2022-05-05,2022-05-05,1
54598,2022-05-05,2022-05-05,1
54602,2022-05-05,2022-05-05,1


# Central

In [46]:
C_RTR = RTR[(RTR['GIS Zone2'].isin(Central_county)) & (~RTR['GIS Zone1'].isin(['NORTHERN DIVISION']))]
s_RTR = RTR[RTR['GIS Zone2'].isin([SCentral_county]) & (~RTR['GIS Zone1'].isin(['NORTHERN DIVISION']))]


Bop_RTR = RTR[RTR['GIS Zone2'].isin([BOP_county]) & (~RTR['GIS Zone1'].isin(['CENTRAL DIVISION']))]
Anvil_RTR = RTR[RTR['GIS Zone2'].isin([Anvil_county]) & (~RTR['GIS Zone1'].isin(['CENTRAL DIVISION']))]

In [13]:
C_RTR['Service Code'].size()

TypeError: 'int' object is not callable

In [48]:
c = C_RTR.groupby(C_RTR['Ticket Date']).agg({'Service Code':'size', 'Total Days on Property': 'mean'}).reset_index()#.to_excel('Central Tree Counts.xlsx')
# s_RTR.groupby(s_RTR['Ticket Date'].dt.date).size().reset_index(name='Count').to_excel('South Central Tree Counts.xlsx')


# Bop_RTR.groupby(Bop_RTR['Ticket Date'].dt.date).size().reset_index(name='Count').to_excel('BOP Tree Counts.xlsx')
# Anvil_RTR.groupby(Anvil_RTR['Ticket Date'].dt.date).size().reset_index(name='Count').to_excel('Anvil Tree Counts.xlsx')


In [11]:
c

,Ticket Date,Count
0,2021-12-08,74
1,2021-12-09,54
2,2021-12-10,78
3,2021-12-11,47
4,2021-12-17,38
...,...,...
104,2022-04-30,328
105,2022-05-02,353
106,2022-05-03,342
107,2022-05-04,480
